In [ ]:
# pip install sentence-transformers faiss-cpu transformers langchain beautifulsoup4 requests python-docx PyMuPDF accelerate

In [1]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import requests
import io
from PyPDF2 import PdfReader
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [2]:
def load_docs():
    # url = "https://dn790007.ca.archive.org/0/items/atomic-habits-pdfdrive/Atomic%20habits%20%28%20PDFDrive%20%29.pdf"
    url = 'https://obssr.od.nih.gov/sites/obssr/files/Clinical-Trials.pdf'
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    if response.status_code != 200:
        print(f"Failed to fetch PDF. Status code: {response.status_code}")
        return []
    reader = PdfReader(io.BytesIO(response.content))
    docs = []
    for page in reader.pages:
        text = page.extract_text()
        if text:
            docs.append(text.strip())
    return docs


In [3]:

# 2. Generate embeddings
embed_model = SentenceTransformer('BAAI/bge-small-en-v1.5')

docs = load_docs()
if not docs:
    raise ValueError("No docs loaded! Check PDF loading.")
embeddings = embed_model.encode(docs, convert_to_tensor=False, show_progress_bar=True)
embeddings = np.stack(embeddings).astype('float32')
print(f'Loaded {len(docs)} pages, Embedding dim: {embeddings.shape[1]}')

# 3. Store in FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)
doc_store = {i: doc for i, doc in enumerate(docs)}


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded 51 pages, Embedding dim: 384


In [4]:
# 4. Fast Retriever function
def retrieve_top_k(query, k=2):
    query_emb = embed_model.encode([query])[0].astype('float32')
    D, I = index.search(np.array([query_emb]), k)
    return [doc_store[i] for i in I[0]]


In [5]:
# pip install -U bitsandbytes

In [6]:
# 5. Load fast, quantized LLM
# Use TinyLlama for fast results (much smaller than Mistral 7B)
llm_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(llm_name)
bnb_config = BitsAndBytesConfig(load_in_4bit=True)  # 4bit quantization for best speed
model = AutoModelForCausalLM.from_pretrained(
    llm_name, torch_dtype=torch.float16, device_map="auto"
)


In [7]:
# prompt = f"Given the following context, answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"


In [8]:
def generate_answer(query):
    context_chunks = retrieve_top_k(query, k=2)
    context = "\n".join(context_chunks)
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=5000, do_sample=True, top_k=40, temperature=0.7)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the FIRST answer after "Answer:" and before another "Question:"
    answer = answer.split("Answer:",1)[-1]
    # Stop at first "Question:" if it appears
    answer = answer.split("Question:")[0]
    return answer.strip()


In [10]:
if __name__ == "__main__":
    query = "Explain phase 3 clinical trials"
    answer = generate_answer(query)

    html_output = f"""
    <div style="font-family: Arial, sans-serif; line-height: 1.6;">
        <h2>Question:</h2>
        <p>{query}</p>
        <h2>Answer:</h2>
        <p>{answer}</p>
    </div>
    """
    print(html_output)



    <div style="font-family: Arial, sans-serif; line-height: 1.6;">
        <h2>Question:</h2>
        <p>Explain phase 3 clinical trials</p>
        <h2>Answer:</h2>
        <p>Phase 3 clinical trials are the final phase of clinical trials, where a new therapeutic 
agent is tested in a large patient population against a placebo or standard therapy. This stage 
assesses the drug's safety, tolerance, and efficacy. The results of this study are used to 
determine if the drug is safe and effective for use in clinical practice. Phase 3 clinical trials are 
important because they provide the foundation for the FDA's approval of new drugs.</p>
    </div>
    
